In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import numpy as np


class GraphConvolutionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvolutionLayer, self).__init__()
        self.weights = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.bias = nn.Parameter(torch.FloatTensor(output_dim))

    def forward(self, x, adjacency):
        x = torch.matmul(x, self.weights)
        x = torch.matmul(adjacency, x)
        x = x + self.bias
        return x


class GraphConvolutionalNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphConvolutionalNetwork, self).__init__()
        self.gc1 = GraphConvolutionLayer(input_dim, hidden_dim)
        self.gc2 = GraphConvolutionLayer(hidden_dim, output_dim)

    def forward(self, x, adjacency):
        x = F.relu(self.gc1(x, adjacency))
        x = self.gc2(x, adjacency)
        return x


# Create a sample graph using NetworkX
graph = nx.karate_club_graph()

# Generate adjacency matrix
adjacency = nx.adjacency_matrix(graph)
adjacency = torch.tensor(adjacency.todense(), dtype=torch.float32)

# Generate node features
features = np.eye(graph.number_of_nodes(), dtype=np.float32)
features = torch.tensor(features, dtype=torch.float32)

# Define model parameters
input_dim = features.shape[1]
hidden_dim = 16
output_dim = 2

# Create GCN model
model = GraphConvolutionalNetwork(input_dim, hidden_dim, output_dim)

# Perform forward pass
output = model(features, adjacency)
print("Output shape:", output.shape)


Output shape: torch.Size([34, 2])
